<a href="https://colab.research.google.com/github/patternproject/p.FlaskDemo/blob/master/flaskDemo_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask Setup to run from colab

In [1]:
import socket
hostname = socket.gethostname()
hostname

'4567863f8540'

In [2]:
ip_address = socket.gethostbyname(hostname)
ip_address

'172.28.0.2'

In [0]:
from flask import Flask
app = Flask(__name__)

In [0]:
@app.route("/")
def hello():
    return "Hello Packt!"

In [5]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [0]:
import threading
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80})	

In [7]:
flask_thread.start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


In [8]:
import requests
r = requests.get("http://172.28.0.2")

172.28.0.2 - - [07/Apr/2020 07:25:58] "GET / HTTP/1.1" 200 -


This log implies that we called the server with IP address 172.28.0.2 on a specific datetime, we sent an HTTP GET request (version 1.1), and the returned status code is 200, which means it was successful.

In [9]:
r.text

'Hello Packt!'

### Adding a POST API Endpoint

In [10]:
r = requests.post("http://172.28.0.2")

172.28.0.2 - - [07/Apr/2020 07:32:14] "POST / HTTP/1.1" 405 -


the log returns a status code of 405, which means there is an error from the client side stating that the POST method is not allowed.

This is because when we declared the root endpoint, we didn't specify any HTTP request type, so by default Flask allows only GET requests. To specify which request type an endpoint can process, we just need to provide the list of request types to the methods parameter of the app.route() decorator. Let's create a new endpoint that accepts both GET and POST requests:

In [0]:
@app.route('/test', methods=['GET', 'POST'])
def test():
  return "Hello Test!"

Now let's try to send a GET request to this new endpoint:

ToDebug: changed the endpoint from `/foo (returning Hello foo)` to `/test (returning Hello Test)` but still ouput has not changed

In [19]:
r_get = requests.get("http://172.28.0.2/test")
r_get.text

172.28.0.2 - - [07/Apr/2020 07:36:36] "GET /test HTTP/1.1" 200 -


'Hello Foo!'

In [20]:
r_get = requests.get("http://172.28.0.2/foo")
r_get.text

172.28.0.2 - - [07/Apr/2020 07:37:06] "GET /foo HTTP/1.1" 200 -


'Hello Foo!'

Let's do the same but with a POST request:

In [21]:
r_post = requests.post("http://172.28.0.2/foo")
r_post.text

172.28.0.2 - - [07/Apr/2020 07:37:40] "POST /foo HTTP/1.1" 200 -


'Hello Foo!'

Great! Now the POST request is working. At the moment, it is just behaving like a GET request. Let's send some data to the endpoint. But first, let's add a new endpoint that only accepts POST requests and returns the same data it receives. In the endpoint function, we need to read the data from the request using the request.get_json() method from Flask. This method will return a dictionary. HTTP requests need to return HTTP-compatible text format. The most popular one is JSON (JSON was mentioned in the first chapter). It is very similar to a Python dictionary.

The jsonify function from Flask will convert a Python dictionary to JSON:

In [0]:
from flask import jsonify, request

@app.route('/display', methods=['POST'])
def print_item():
  data = request.get_json()
  return jsonify(data)

Now we have to send data with the POST request. First, let's create a list of data we will send to this endpoint:

In [0]:
data = ['Australia', 'France', 'China']

We need to convert it into JSON. We will use the .dumps() method from the json package:

In [0]:
import json
j_data = json.dumps(data)

To send data via a POST request, we need to provide the type of information sent. We will provide a dictionary to the header parameter of the POST request: headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}. This dictionary specifies the type of data. Here it will be JSON, and the text encoding is UTF-8:

In [25]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

r = requests.post("http://172.28.0.2/display", data=j_data, headers=headers)

172.28.0.2 - - [07/Apr/2020 07:40:46] "POST /display HTTP/1.1" 200 -


Our POST request to the display/ endpoint was successful, as shown by the logs. Now let's print the response received from the server by looking at the .text attribute:

In [26]:
r.text

'["Australia","France","China"]\n'